In [1]:
import os
from sklearn.feature_extraction.text import CountVectorizer
import lda
import numpy as np
import pickle
import seaborn as sns

from matplotlib import pyplot as plt 

In [2]:
root = "/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/"

In [3]:
filename = root + "done_processing.txt"
rating_file = root + "rating.txt"
source = root + "source.txt"

source_name = []
corpus = []
page_id = []
doc_count = 0
rating = []
blank = []

with open(filename, 'r') as f:
    line = f.readline()
    while(line):
        if(line[0]!="*"):
            corpus.append(line)
        elif(lens(line)==0):
            print("wtf?")
            # doc_count = doc_count +1
        line = f.readline()
with open(rating_file, 'r') as f2:
    line = f2.readline()
    while(line):
        if(line[0]=="*"):
            page_id.append(doc_count)#-len(page_id))
        else:
            rating.append(np.int64(line))
            doc_count = doc_count +1
        line = f2.readline()
with open(source, 'r') as f3:
    line = f3.readline()
    while(line):
        if(line[0]=="*"):
#             page_id.append(doc_count-len(page_id))
            a = 6
        else:
            source_name.append(line)
            doc_count = doc_count +1
        line = f3.readline()
#         print(line)


for i in range(len(page_id)-1):
    sum = 0
    for j in range(page_id[i],page_id[i+1]):
        sum = sum + rating[j]
        # print(j)
    for j in range(page_id[i],page_id[i+1]):
        rating[j] = rating[j]/sum

sum = 0
for j in range(page_id[-1],len(rating)):
    sum = sum + rating[j]
for j in range(page_id[-1],len(rating)):
    rating[j] = rating[j]/sum
# print(rating)
# print(corpus)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
vocab = vectorizer.get_feature_names()
# print(vectorizer.get_feature_names())
# print(X.toarray())

model_filename = root+'finalized_model.sav'
model = pickle.load(open(model_filename, 'rb'))

page_vector = []
topic_word = model.topic_word_
doc_topic = model.doc_topic_
for i in range(len(page_id)-1):
    vector_temp = np.zeros((20,))
    for j in range(page_id[i],page_id[i+1]):
        vector_temp = vector_temp + rating[j]*doc_topic[j]
    page_vector.append(vector_temp)

vector_temp = np.zeros((1, 20))
print(len(rating))
for j in range(page_id[-1],len(corpus)):
    vector_temp = vector_temp + rating[j]*doc_topic[j]
page_vector.append(vector_temp)



27227


In [4]:
class Fanpage:
    def __init__(self, name, corpus, ratings, word_vectors):
        self.name = name
        self.corpus = corpus
        self.ratings = ratings
        self.word_vectors = word_vectors
        self.page_vector = np.zeros((20,))
    def calculate(self):
        sum = np.zeros((20,))
        for i in range(len(self.ratings)):
            sum = sum + self.ratings[i]*self.word_vectors[i]
        if(len(self.ratings)<=1):
            self.page_vector = np.zeros((20,))
        else:
            self.page_vector = sum
        

In [5]:
set_page = []
print(len(corpus))
print(len(page_id))
for i in range(len(page_id)):
    name = source_name[i]
    t_corpus = []
    t_ratings = []
    t_word_vectors = []
    if(i<len(page_id)-1):
        for j in range(page_id[i],page_id[i+1]):
            t_corpus.append(corpus[j])
            t_ratings.append(rating[j])
            t_word_vectors.append(doc_topic[j])
    else:
        for j in range(page_id[i],len(corpus)):
#             print(j)
            t_corpus.append(corpus[j])
            t_ratings.append(rating[j])
            t_word_vectors.append(doc_topic[j])
    print(name,len(t_ratings))
    page = Fanpage(name,t_corpus,t_ratings,t_word_vectors)
    set_page.append(page)


27226
91
YAN News
 393
Đại Kỷ Nguyên,Đại Kỷ Nguyên,Tin tức kịp thời
 314
YAN TV
 420
YAN News
 396
Đại Kỷ Nguyên,Đại Kỷ Nguyên,Thời sự kịp thời
 322
YAN TV
 417
Yeah1 TV
 190
Đại Kỷ Nguyên - News
 221
Kenh14.vn
 470
Vie Channel - HTV2
 300
VnExpress.net
 487
Man TV
 235
Đại Kỷ Nguyên - Video,Đại Kỷ Nguyên,Thời sự kịp thời
 480
VOA Tiếng Việt
 470
BBC News Tiếng Việt
 333
Tuổi Trẻ
 530
Muvik
 11
YAN Foods
 309
Trung tâm Tin tức VTV24
 419
TodayTV
 379
Trắng TV
 305
Báo Thanh Niên
 61
SaoStar
 503
VN Ngày Nay
 206
Thethao247.vn
 479
VTC News
 502
POPS Kids
 306
YAN KPOP,YAN TV
 412
VTV Giải trí,VTV6
 326
YouTV
 396
BEATVN
 421
Truyền hình FPT
 286
Hoa Hoc Tro Magazine
 315
Báo Đời Sống Pháp Luật
 232
Đài Á Châu Tự Do
 296
HTV3 - DreamsTV
 294
TinMoi.vn
 310
ELLE Vietnam
 342
Kênh Nhạc Việt
 315
Marry
 10
Xone Radio
 304
VTV - Đài Truyền hình Việt Nam
 371
Bà Lão - Trị Mụn Thâm Sẹo Nám
 8
Tạp Chí Đẹp
 305
Giải Trí TV
 314
THVL - Đài Phát thanh Truyền hình Vĩnh Long
 511
VN Ngày Nay - Tin 

In [6]:
for page in set_page:
    page.calculate()

In [7]:
from sklearn.cluster import KMeans
# kmeans = KMeans(n_clusters=3, random_state=0).fit(X)

In [8]:
vector_pages = []
for page in set_page:
    vector_pages.append(page.page_vector)
    print(page.page_vector)

[0.1005176  0.0289949  0.02161144 0.01915381 0.0253015  0.01641976
 0.02131784 0.02748174 0.05635025 0.01719351 0.16933776 0.12519122
 0.02105463 0.02896859 0.07853545 0.03198551 0.08356061 0.01589233
 0.04421927 0.06691227]
[0.01408108 0.01741227 0.04254234 0.02102277 0.0167857  0.02229079
 0.01350413 0.02006484 0.09345417 0.01141558 0.21294264 0.19340159
 0.02467508 0.11740718 0.02816507 0.0160006  0.03394614 0.02624099
 0.01853116 0.05611588]
[0.08394391 0.0220951  0.01541927 0.03649962 0.02034548 0.02290405
 0.0354213  0.02398621 0.0406581  0.01937052 0.09841265 0.11756324
 0.0362604  0.02348256 0.07926272 0.03808813 0.13994377 0.03100357
 0.0534826  0.06185683]
[0.09754567 0.02868746 0.02341492 0.0219561  0.02495982 0.0166161
 0.03050702 0.02685589 0.0532079  0.02063112 0.15666895 0.13439723
 0.02542456 0.02264343 0.07417367 0.02931266 0.0906586  0.01460225
 0.04401812 0.06371852]
[0.01296569 0.01686767 0.03515843 0.02445437 0.0242592  0.02079657
 0.01237355 0.02800493 0.08557128 

[0.01425707 0.01586996 0.02102969 0.04203249 0.01835321 0.01040692
 0.03216899 0.0090704  0.01435376 0.01130933 0.06361833 0.04073842
 0.36572099 0.05981217 0.04400758 0.02824301 0.0247186  0.02404922
 0.01410266 0.1461372 ]
[0.02756025 0.02943945 0.02706458 0.02366664 0.03813446 0.02677344
 0.28247622 0.0758614  0.11308931 0.02307684 0.019871   0.01936904
 0.03511657 0.03567835 0.03773566 0.0358847  0.02778063 0.02304783
 0.06569997 0.03267366]
[0.03501156 0.02219484 0.01503048 0.43537168 0.01710538 0.01675435
 0.01304925 0.01671338 0.01234879 0.01203524 0.02666836 0.10100311
 0.03027078 0.01822408 0.08680946 0.01321744 0.07435147 0.01450454
 0.01203024 0.02730557]
[0.27538735 0.01613499 0.02913957 0.01070473 0.01917773 0.01592433
 0.13413861 0.01671896 0.02556402 0.16182102 0.04235336 0.05506319
 0.02489598 0.02105962 0.02221364 0.01574074 0.03157788 0.00816689
 0.04188961 0.03232775]
[0.0260982  0.03554261 0.0205576  0.0319641  0.01371801 0.01539335
 0.4346226  0.01969506 0.06566235

In [9]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(vector_pages)

In [10]:
pred_label = kmeans.predict(vector_pages)


In [11]:
import matplotlib.pyplot as plt 
num = 4
set_t = []
# for i in range(len(pred_label)):
#     if(pred_label[i]==num):
# #         print(set_page[i].name)
#         set_t.append(set_page[i])

for i,page in enumerate(set_page):
    plt.figure()
    plt.plot(page.page_vector)
    plt.xlabel('Topic')
    plt.ylabel('Probability')
    plt.xticks(np.arange(0, 20, 1.0))
    plt.yticks(np.arange(0, 0.8, 0.1))
    plt.savefig(root+"fig/"+str(pred_label[i])+"-"+page.name+".png")
    print(root+"fig/"+str(pred_label[i])+"-"+page.name+".png")
    plt.close()

/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/fig/3-YAN News
.png
/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/fig/3-Đại Kỷ Nguyên,Đại Kỷ Nguyên,Tin tức kịp thời
.png
/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/fig/3-YAN TV
.png
/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/fig/3-YAN News
.png
/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/fig/3-Đại Kỷ Nguyên,Đại Kỷ Nguyên,Thời sự kịp thời
.png
/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/fig/3-YAN TV
.png
/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/fig/4-Yeah1 TV
.png
/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/fig/3-Đại Kỷ Nguyên - News
.png
/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/fig/3-Kenh14.vn
.png
/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/fig/1-Vie Channel - HTV2
.png
/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/fig/3-VnExpress.net
.png
/home/vietphan/Downloads/fbcrawl/Data-Media-Oct/model/fig/3-Man TV
.png
/home/vietphan/Dow